# Proposition 7 Analysis

### Unified Model, Multiple Weighted Strategic Agents, Symmetric Network

James Yu, 20 June 2023

In [1]:
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(suppress=True)

In [2]:
def M(K, B, R, L, delta):
    """Computes M_{t-1} given B_l \forall l, K_t^l \forall l, 
        R_l \forall l, number of strategic agents L, and delta."""
    # handle the generic structure first, with the correct pairings:
    base = [[B[l_prime].T @ K[l_prime] @ B[l] for l in range(L)] for l_prime in range(L)]
    # then change the diagonals to construct M_{t-1}:
    for l in range(L): base[l][l] = B[l].T @ K[l] @ B[l] + R[l]/delta
    return np.block(base)

def H(B, K, A, L):
    """Computes H_{t-1} given B_l \forall l, K_t^l \forall l, 
        A, and number of strategic agents L."""
    return np.concatenate(tuple(B[l].T @ K[l] @ A for l in range(L)), axis = 0)

def C_l(A, B, K, k, h, L, c, x, n):
    """Computes C_{t-1}^h (displayed as C_{t-1}^l) given A, B_l \forall l, K_t^l \forall l, 
        k_t^l \forall l, a specific naive agent h, number of strategic agents L, 
        c_l \forall l, x_l \forall l, and number of naive agents n"""
    return np.concatenate(tuple(B[l].T @ K[l] @ A @ ((x[h] - x[l]) * np.ones((n, 1))) 
                           + B[l].T @ K[l] @ c[l] 
                           + 0.5 * B[l].T @ k[l].T for l in range(L)), axis = 0)

def E(M_, H_):
    """Computes the generic E_{t-1} given M_{t-1} and H_{t-1}."""
    return np.linalg.inv(M_) @ H_

def F(M_, C_l_, l, n):
    """Computes F_{t-1}^l given M_{t-1}, C_{t-1}^l, 
       specific naive agent l and number of naive agents n."""
    return (np.linalg.inv(M_) @ C_l_)[l*n:(l+1)*n, :] # e.g. l = 0 gives ln = 0, l = 1 gives ln = n, etc

def G(A, B, E_, L, n):
    """Computes the generic G_{t-1} given A, B_l \forall l, 
        E_{t-1}, number of strategic agents L, and number of naive agents n."""
    return A - sum([B[l] @ E_[l*n:(l+1)*n, :] for l in range(L)])
    
def g_l(B, E_, h, x, F_, L, n, c):
    """Computes g_{t-1}^l given B_l \forall l, E_{t-1}^l, 
        a particular naive agent h, x_l \forall l, F_{t-1}^l \forall l, 
        number of strategic agents L, number of naive agents n, and c_h."""
    return - sum([B[l] @ (E_[l*n:(l+1)*n, :] @ ((x[h] - x[l]) * np.ones((n, 1))) + F_[l]) for l in range(L)]) + c[h]

In [3]:
def K_t_minus_1(Q, K, E_, R, G_, L, delta, n):
    return [Q[l] + E_[l*n:(l+1)*n, :].T @ R[l] @ E_[l*n:(l+1)*n, :] 
            + delta * G_.T @ K[l] @ G_ for l in range(L)]

def k_t_minus_1(K, k, G_, g, E_, F_, R, L, delta, n):
    return [2*delta* g[l].T @ K[l] @ G_ + delta * k[l] @ G_ 
            + 2 * F_[l].T @ R[l] @ E_[l*n:(l+1)*n, :] for l in range(L)]

def kappa_t_minus_1(K, k, kappa, g_, F_, R, L, delta):            
    return [-delta * (g_[l].T @ K[l] @ g_[l] + k[l] @ g_[l] - kappa[l]) 
            - (F_[l].T @ R[l] @ F_[l]) for l in range(L)]

In [4]:
def should_terminate(bundles, eps):
    return all([np.allclose(b[0], b[1], rtol = eps, atol = eps) for b in bundles])

def solve(K_t, k_t, kappa_t, A, B, delta, n, L, Q, R, x, c, tol):
    historical_K = [K_t]
    historical_k = [k_t]
    historical_kappa = [kappa_t]
    eps = np.sqrt(np.finfo(np.float64).eps)
    while True:
        M_ = M(K_t, B, R, L, delta)
        H_ = H(B, K_t, A, L)
        E_ = E(M_, H_)
        G_ = G(A, B, E_, L, n)
        K_new = K_t_minus_1(Q, K_t, E_, R, G_, L, delta, n)
        F_ = [F(M_, C_l(A, B, K_t, k_t, l, L, c, x, n), l, n) for l in range(L)]
        g = [g_l(B, E_, h, x, F_, L, n, c) for h in range(L)]
        k_new = k_t_minus_1(K_t, k_t, G_, g, E_, F_, R, L, delta, n)
        kappa_new = kappa_t_minus_1(K_t, k_t, kappa_t, g, F_, R, L, delta)
        historical_K.insert(0, K_new)
        historical_k.insert(0, k_new)
        historical_kappa.insert(0, kappa_new)
        if should_terminate([(K_t, K_new), (k_t, k_new), (kappa_t, kappa_new)], eps):
            return historical_K[0], historical_k[0], historical_kappa[0]
        K_t = K_new
        k_t = k_new
        kappa_t = kappa_new

In [5]:
def optimal(X_init, K_ss, k_ss, A, B, delta, n, L, Q, R, x, c, eps):
    X_t = [a.copy() for a in X_init]
    xs = defaultdict(list)
    for l in range(L):
        xs[l].append(X_t[l])
        
    rs = defaultdict(list)
    payoffs = defaultdict(list)
    payoff = defaultdict(int)
    
    M_ = M(K_ss, B, R, L, delta)
    H_ = H(B, K_ss, A, L)
    E_ = E(M_, H_)
    G_ = G(A, B, E_, L, n)
    F_ = [F(M_, C_l(A, B, K_ss, k_ss, l, L, c, x, n), l, n) for l in range(L)]
    g = [g_l(B, E_, h, x, F_, L, n, c) for h in range(L)]
    
    i = 0
    while True:
        for l in range(L):
            # TODO: special code for finite horizon needs a terminal time T term
            Y_new = -1 * E_[l*n:(l+1)*n, :] @ X_t[l] - F(M_, C_l(A, B, K_ss, k_ss, l, L, c, x, n), l, n)
            rs[l].append(Y_new)
            payoff[l] += (-1 * delta**i * (X_t[l].T @ Q[l] @ X_t[l])).item() + (-1 * delta**i * (Y_new.T @ R[l] @ Y_new)).item()
            payoffs[l].append(payoff[l])
            X_new = G_ @ X_t[l] + g[l]
            xs[l].append(X_new)
            if l == L - 1 and np.allclose(X_t[l], X_new, rtol = eps, atol = eps):
                return xs, rs, payoffs
            X_t[l] = X_new 
        i += 1
        
    return xs, rs, payoffs

In [6]:
def run_simulation(A, b1, b2, a1, a2, delta, c, X_0_1, tol = np.sqrt(np.finfo(np.float64).eps)):
    X_0 = [X_0_1 - b1, X_0_1 - b2]
    n = A.shape[0] # number of naive agents
    L = 2 # number of strategic agents
    Q = [np.identity(n), np.identity(n)] # objective function for messages is just X'IX = X'X
    R = [c * np.identity(n), c * np.identity(n)] # message cost R = cI_n for some c under the new notation
    B = [a1 * np.identity(n), a2 * np.identity(n)] # B^l = a_l I_n
    x = [b1, b2] # agendas
    r = [0, 0] # message cost minimality is centered around zero
    c_base = sum([B[l] @ (r[l] * np.ones((n, 1))) for l in range(L)])
    c = [c_base + (A - np.identity(n)) @ (x[l] * np.ones((n, 1))) for l in range(L)] # normalization vector
    
    K_ss, k_ss, kappa_ss = solve(Q, [np.zeros((1, n)), np.zeros((1, n))], [0, 0], A, B, delta, n, L, Q, R, x, c, tol)
    xs, rs, payoffs = optimal(X_0, K_ss, k_ss, A, B, delta, n, L, Q, R, x, c, tol)
    return K_ss, k_ss, kappa_ss, xs, rs, payoffs

In [7]:
A1 = np.array([
    [0.3, 0.5, 0.2],
    [0.5, 0.4, 0.1],
    [0.2, 0.1, 0.7]
])

delta = 0.9
c = 1.0
X_0_1 = np.array([[10.0, -5.0, 5.0]], ndmin = 2).T
b1 = 10
b2 = -10
a1 = 6
a2 = 5
K_ss, k_ss, kappa_ss, xs, rs, payoffs = run_simulation(A1, b1, b2, a1, a2, delta, c, X_0_1)

## 43, 44

In [8]:
from scipy.optimize import fsolve

In [9]:
eigvals_raw, U_raw = np.linalg.eig(A1)
# sort eigenvalues largest to smallest so the "1" comes first
eigval_sorting_map = np.flip(np.argsort(eigvals_raw)) 
eigvals = eigvals_raw[eigval_sorting_map]
U = U_raw[:, eigval_sorting_map]

In [10]:
eigval_sorting_map

array([1, 2, 0], dtype=int64)

In [11]:
eigvals_raw, U_raw

(array([-0.16055513,  1.        ,  0.56055513]),
 array([[ 0.75130448, -0.57735027, -0.31970025],
        [-0.65252078, -0.57735027, -0.49079864],
        [-0.0987837 , -0.57735027,  0.81049889]]))

In [12]:
eigvals, U

(array([ 1.        ,  0.56055513, -0.16055513]),
 array([[-0.57735027, -0.31970025,  0.75130448],
        [-0.57735027, -0.49079864, -0.65252078],
        [-0.57735027,  0.81049889, -0.0987837 ]]))

In [13]:
D = np.diag(eigvals)

In [14]:
def tilde_Kj(c, delta, a1, a2, lambdaj):
    g1 = lambda x, y: x - (1 + (c*delta * lambdaj**2 * x * (delta*x*a1**2 + c))/((c+delta*x*a1**2 + delta*y*a2**2)**2))
    g2 = lambda x, y: y - (1 + (c*delta * lambdaj**2 * y * (delta*y*a2**2 + c))/((c+delta*x*a1**2 + delta*y*a2**2)**2))
    g = lambda x: (g1(x[0], x[1]), g2(x[0], x[1]))
    return fsolve(g, [1, 1])

In [15]:
tilde_K_constructor = [tilde_Kj(c, delta, a1, a2, lambdaj) for lambdaj in eigvals]
tilde_K_constructor

[array([1.00964258, 1.00674558]),
 array([1.00302499, 1.00212454]),
 array([1.00024799, 1.00017446])]

In [16]:
tilde_Kss_1 = np.diag([K[0] for K in tilde_K_constructor])
tilde_Kss_1

array([[1.00964258, 0.        , 0.        ],
       [0.        , 1.00302499, 0.        ],
       [0.        , 0.        , 1.00024799]])

In [17]:
tilde_Kss_2 = np.diag([K[1] for K in tilde_K_constructor])
tilde_Kss_2

array([[1.00674558, 0.        , 0.        ],
       [0.        , 1.00212454, 0.        ],
       [0.        , 0.        , 1.00017446]])

In [18]:
np.allclose(K_ss[0], U @ tilde_Kss_1 @ U.T)

True

In [19]:
np.allclose(K_ss[1], U @ tilde_Kss_2 @ U.T)

True

## 41, 42

In [20]:
I = np.identity(3)
Gamma = np.linalg.inv(c*I + delta*a1**2 * tilde_Kss_1 + delta*a2**2 * tilde_Kss_2)
np.allclose(tilde_Kss_1, I + c*delta*D @ Gamma @ tilde_Kss_1 @ (delta*a1**2 * tilde_Kss_1 + c*I) @ Gamma @ D)

True

In [21]:
np.allclose(tilde_Kss_2, I + c*delta*D @ Gamma @ tilde_Kss_2 @ (delta*a2**2 * tilde_Kss_2 + c*I) @ Gamma @ D)

True

## 50, 51

In [22]:
Y = I - delta * Gamma @ Gamma @ D @ (delta * a1**2 * tilde_Kss_1 + c*I) @ (delta * a2**2 * tilde_Kss_2 + c*I)
Z1 = delta**2 * a2**2 * Gamma @ D @ (delta * a1**2 * tilde_Kss_1 + c*I) @ tilde_Kss_1 @ Gamma
Z2 = delta**2 * a1**2 * Gamma @ D @ (delta * a2**2 * tilde_Kss_2 + c*I) @ tilde_Kss_2 @ Gamma
tilde_b1 = U.T @ (b1 * np.ones((3, 1)))
tilde_b2 = U.T @ (b2 * np.ones((3, 1)))
tilde_Kk1 = np.linalg.inv((Y @ Y) - (Z1 @ Z2)) @ ((Y @ tilde_b1) - (Z1 @ tilde_b2))
tilde_Kk1

array([[-31.10691353],
       [  0.        ],
       [  0.        ]])

In [23]:
tilde_Kk2 = np.linalg.inv(Y @ Y - Z1 @ Z2) @ (Y @ tilde_b2 - Z2 @ tilde_b1)
tilde_Kk2

array([[31.15709111],
       [-0.        ],
       [-0.        ]])

In [24]:
U @ tilde_Kk1

array([[17.9595849],
       [17.9595849],
       [17.9595849]])

In [25]:
U @ tilde_Kk2

array([[-17.98855494],
       [-17.98855494],
       [-17.98855494]])

In [26]:
K_ss[0] @ k_ss[0].T

array([[-15.87796046],
       [-15.87796046],
       [-15.87796046]])

In [27]:
K_ss[1] @ k_ss[1].T

array([[15.94906308],
       [15.94906308],
       [15.94906308]])

TODO: why does this not match?

## 48, 49

In [28]:
w = a1**2 * tilde_Kk1 + a2**2 * tilde_Kk2
np.allclose(tilde_Kk1, tilde_b1 - delta * Gamma @ D @ (delta * a1**2 * tilde_Kss_1 + c*I) @ (delta * tilde_Kss_1 @ Gamma @ w - tilde_Kk1))

True

In [29]:
np.allclose(tilde_Kk2, tilde_b2 - delta * Gamma @ D @ (delta * a2**2 * tilde_Kss_2 + c*I) @ (delta * tilde_Kss_2 @ Gamma @ w - tilde_Kk2))

True

## Limit Opinions

In [30]:
x_ss = xs[0][-1]+b1
x_ss

array([[3.19968643],
       [3.19968643],
       [3.19968643]])

In [31]:
U.T @ x_ss

array([[-5.54201947],
       [ 0.        ],
       [-0.        ]])

In [32]:
np.linalg.inv(I - c*Gamma @ D) @ (delta * Gamma) @ w

array([[-5.54201947],
       [ 0.        ],
       [ 0.        ]])

In [33]:
np.linalg.inv(a1**2 * tilde_Kss_1 + a2**2 * tilde_Kss_2) @ w

array([[-5.54201947],
       [ 0.        ],
       [ 0.        ]])

## 52

In [34]:
(a1**2 * tilde_Kss_1 + a2**2 * tilde_Kss_2) @ (Y@Y - Z1@Z2) @ (U.T @ x_ss)

array([[-188.13233947],
       [   0.00000002],
       [  -0.        ]])

In [35]:
a1**2 * (I - delta*Gamma @ D @ (delta * a2**2 * tilde_Kss_2 + c*I))@tilde_b1 + a2**2 * (I - delta*Gamma@D@(delta*a1**2 * tilde_Kss_1 + c*I))@tilde_b2

array([[-188.13233945],
       [   0.        ],
       [   0.        ]])

In [36]:
(Y@Y - Z1@Z2)@w

array([[-188.13233945],
       [   0.        ],
       [   0.        ]])

## b1 > b2

In [37]:
K_ss, k_ss, kappa_ss, xs, rs, payoffs = run_simulation(A1, b1, b2, a1, a2, delta, c, X_0_1)
np.abs(U.T @ rs[0][-1]) # influencer 1

array([[137.76185518],
       [  0.        ],
       [  0.        ]])

In [38]:
np.abs(U.T @ rs[1][-1]) # influencer 2

array([[165.31422623],
       [  0.        ],
       [  0.        ]])

In [39]:
K_ss, k_ss, kappa_ss, xs, rs, payoffs = run_simulation(A1, 10, 9, a1, a2, delta, c, X_0_1)
np.abs(U.T @ rs[0][-1]), np.abs(U.T @ rs[1][-1])

(array([[6.88809276],
        [0.        ],
        [0.        ]]),
 array([[8.26571131],
        [0.        ],
        [0.        ]]))

In [40]:
K_ss, k_ss, kappa_ss, xs, rs, payoffs = run_simulation(A1, 10, 11, a1, a2, delta, c, X_0_1)
np.abs(U.T @ rs[0][-1]), np.abs(U.T @ rs[1][-1])

(array([[6.88809276],
        [0.        ],
        [0.        ]]),
 array([[8.26571131],
        [0.        ],
        [0.        ]]))

In [41]:
K_ss, k_ss, kappa_ss, xs, rs, payoffs = run_simulation(A1, 10, 10, a1, a2, delta, c, X_0_1)
np.abs(U.T @ rs[0][-1]), np.abs(U.T @ rs[1][-1])

(array([[0.],
        [0.],
        [0.]]),
 array([[0.],
        [0.],
        [0.]]))

In [42]:
payoffs[0][-1]

-251.3450261100448

In [43]:
payoffs[1][-1]

-250.9411231791053